In [47]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import numpy as np
import scipy.io as sio
import scipy.interpolate
import matplotlib.pyplot as plt
import os

In [49]:
#Update dataset name as needed

# EXPERIMENT = "sand"
EXPERIMENT = "clay"

# RAW_DATA_FOLDER_CLAY = '../'
# RAW_MAT_FOLDER_CLAY = RAW_DATA_FOLDER_CLAY + 'raw_mat_new_clay/'
# RAW_NPY_FOLDER_CLAY = RAW_DATA_FOLDER_CLAY + 'raw_npy_clay/'
# NPY_FOLDER_CLAY = '../processed_input_data_clay/'

RAW_DATA_FOLDER = '../'
RAW_MAT_FOLDER = RAW_DATA_FOLDER + 'raw_mat_new_' + EXPERIMENT + '/'
RAW_NPY_FOLDER = RAW_DATA_FOLDER + 'raw_npy_' + EXPERIMENT + '/'
NPY_FOLDER = '../processed_input_data_' + EXPERIMENT + '/'

#split data
## attribute all exp_1 to training
## attribute 2 windows (split_ratio) from exp_2 and exp_3 to training
## the rest were randomly used for either eval and train. 
ML_exp = 'split_master'
split_ratio=2

In [50]:
#EXP_LIST contains a list of raw experiment XXX.mat files 
#Format: raw .mat --> raw .npy --> sliced & labeled .npy 

if EXPERIMENT == "clay":
# Experiments list for Clay
    EXP_LIST = ['EB_025_1','EB_025_2','EB_025_3',
            'EB_050_1','EB_050_2','EB_050_3',
            'EB_150_1','EB_150_2','EB_150_3',
            'PP_025_1','PP_025_2',
            'PP_050_1','PP_050_2',
            'PP_100_1','PP_100_2',
            'PP_150_1']
else:
# Experiments list for Sand
    EXP_LIST = ['E355_64px_forCNN',
    'E339_64px_forCNN',
    'E280_64px_forCNN',
    'E366_64px_forCNN',
    'E342_64px_forCNN',
    'E340_64px_forCNN',
    'E364_64px_forCNN',
    'E347_64px_forCNN',
    'E346_64px_forCNN',
    'E341_64px_forCNN',
    'E365_64px_forCNN',
    'E338_64px_forCNN',
    'E345_64px_forCNN',
    'E363_64px_forCNN',
    'E368_64px_forCNN',
    'E369_64px_forCNN',
    'E458_64px_forCNN',
    'E459_64px_forCNN',
    'E463_64px_forCNN',
    'E464_64px_forCNN',
    'E499_64px_forCNN']


In [51]:
# Basic functions to generate.npy files from mat files 


def create_npy(exp):
    matlab_data_path = RAW_MAT_FOLDER + str(exp) + '.mat' 
    npy_data_path = RAW_NPY_FOLDER + str(exp) +'data.npy'
    npy_label_path = RAW_NPY_FOLDER + str(exp) +'label.npy'
    npy_para_path = RAW_NPY_FOLDER + str(exp) +'para.npy'
    
    if not os.path.exists(RAW_NPY_FOLDER):
        os.mkdir(RAW_NPY_FOLDER)
    if os.path.exists(npy_data_path):
        raise Exception("raw .npy file already exist")
    else:  
        data = sio.loadmat(matlab_data_path) ["fault_map"]  
        label = sio.loadmat(matlab_data_path) ["label"]
        
        WW = sio.loadmat(matlab_data_path) ["windowWidth"]
        overlap = sio.loadmat(matlab_data_path) ["overlap"]
        
        slice_param = [0 for a in range(4)]
        slice_param[3] = WW[0,0]
        slice_param[2] = 2* slice_param[3]
        slice_param[1] = slice_param[3]- overlap[0,0]
        slice_param[0] = 0
        
        # report sizes and save data, parameters and label
        print(data.shape)
        print(label.shape)
        print(slice_param)
        np.save(npy_data_path, data)
        np.save(npy_label_path, label)
        np.save(npy_para_path, slice_param)

In [52]:
# Generate and save raw_npy for list of experiment
# Only need to run once! 

for e in EXP_LIST:
    print(e)
    create_npy(e)

EB_025_1
(128, 264, 227)
(5, 227, 2)
[0, 50, 128, 64]
EB_025_2
(128, 264, 227)
(5, 227, 2)
[0, 50, 128, 64]
EB_025_3
(128, 264, 227)
(5, 227, 2)
[0, 50, 128, 64]
EB_050_1
(128, 264, 218)
(5, 218, 2)
[0, 50, 128, 64]
EB_050_2
(128, 264, 219)
(5, 219, 2)
[0, 50, 128, 64]
EB_050_3
(128, 264, 220)
(5, 220, 2)
[0, 50, 128, 64]
EB_150_1
(128, 264, 192)
(5, 192, 2)
[0, 50, 128, 64]
EB_150_2
(128, 264, 193)
(5, 193, 2)
[0, 50, 128, 64]
EB_150_3
(128, 264, 191)
(5, 191, 2)
[0, 50, 128, 64]
PP_025_1
(128, 264, 227)
(5, 227, 2)
[0, 50, 128, 64]
PP_025_2
(128, 264, 228)
(5, 228, 2)
[0, 50, 128, 64]
PP_050_1
(128, 264, 219)
(5, 219, 2)
[0, 50, 128, 64]
PP_050_2
(128, 264, 218)
(5, 218, 2)
[0, 50, 128, 64]
PP_100_1
(128, 264, 190)
(5, 190, 2)
[0, 50, 128, 64]
PP_100_2
(128, 264, 205)
(5, 205, 2)
[0, 50, 128, 64]
PP_150_1
(128, 264, 204)
(5, 204, 2)
[0, 50, 128, 64]


In [53]:
#This function creates slices of each fault map
def slicing_hw(RAW_NPY_FOLDER, exp): 
    npy_data_path = RAW_NPY_FOLDER + str(exp) +'data.npy'
    npy_para_path = RAW_NPY_FOLDER + str(exp) +'para.npy'
    
    data = np.load(npy_data_path)
    print(data.shape)
    slice_param = np.load(npy_para_path)
    
    #data has dimension of (x, y, cfuv, t)
    width_tot = data.shape[1]
    t_tot = data.shape[2]
    #print(width_tot,t_tot)
    
    #unpack slice_param into initial_index, strike_lenght, window_H and window_W
    ii = slice_param[0]
    strike = slice_param[1]
    H = slice_param[2]
    W = slice_param[3]
    print(ii,strike,H,W)
    
    # set number of windows and slice data and label 
    num_win = 1 + (width_tot - W)//strike   
    
    data_hw = []
    w_idx = np.arange(num_win)
    for w_i in w_idx:
        left_win = w_i*strike
        #print(left_win)
        data_hwi = data[:,left_win:left_win+W]
        #data_hw is a list of tuples (number of tuples = num_win)
        #first element of tuple has a shape = (WindowHeight, WindowWidth, numFrames)
        #second element of tuple is the w_idx
        data_hw.append((data_hwi, w_i))

    print("Data_slice_shape =", data_hw[0][0].shape)
    return data_hw, num_win, t_tot


In [54]:
#This function gathers the label information and saves with slices to be read for dividing into 
#training, eval and testing data sets.
def data_slice_KE(RAW_NPY_FOLDER, NPY_FOLDER, exp):
    
    #set file location for 
    npy_exp_folder_path = NPY_FOLDER + 'slice_npy/' + str(exp)
    master_folder_path = NPY_FOLDER + 'file_master/'
    
    #need to grab the KE labels
    npy_label_path = RAW_NPY_FOLDER + str(exp) +'label.npy'
    label = np.load(npy_label_path)
    #print('label', label.shape)
    
    if os.path.exists(npy_exp_folder_path):
        raise Exception("slice_exp.npy already exist")
    else: 
        os.makedirs(npy_exp_folder_path)
        if not os.path.exists(master_folder_path): 
            os.makedirs(master_folder_path)     
    
    #slice the fault_map
    data_hw, num_win, t_tot = slicing_hw(RAW_NPY_FOLDER, exp)
    
    w_idx = np.arange(num_win)
    file_master = []
    
    #looping through all windows and times 
    for w_i in w_idx: 
        for t_i in range(t_tot):
            #unpacking data_hw
            data_f, win = data_hw[w_i]
            #print(win)
        
            #only process slice with non-zero fault traces
            if np.sum(data_f[:,:,t_i]) != 0:
                
                #The first array of the label array are the KE and the second array is the std
                label_KE = label[w_i,t_i,0]
                label_SD = label[w_i,t_i,1]
                #print(w_i,t_i,label_KE,label_SD)
                
                name_path = '/' + str(label_KE) + '_' + str(label_SD) + '_win_' + str(
                        win) + '_t_' + str('{:03d}'.format(t_i)) + '_' + str(exp) + '_cfuv.npy'
                # save sliced and labeled dataset in slice_npy folder
                np.save(npy_exp_folder_path + name_path, data_f[:,:,t_i])
                
                # add to file master
                file_master.append(str(exp) + name_path)          
    
    master_data_path = master_folder_path + str(exp) + ".txt"
    print('non_zero_slice =', len(file_master))
    np.savetxt(master_data_path, file_master, fmt="%s")

In [56]:
# Only need to run once! 
for exp in EXP_LIST:
    data_slice_KE(RAW_NPY_FOLDER, NPY_FOLDER, exp)

(128, 264, 227)
0 50 128 64
Data_slice_shape = (128, 64, 227)
non_zero_slice = 927
(128, 264, 227)
0 50 128 64
Data_slice_shape = (128, 64, 227)
non_zero_slice = 1005
(128, 264, 227)
0 50 128 64
Data_slice_shape = (128, 64, 227)
non_zero_slice = 931
(128, 264, 218)
0 50 128 64
Data_slice_shape = (128, 64, 218)
non_zero_slice = 892
(128, 264, 219)
0 50 128 64
Data_slice_shape = (128, 64, 219)
non_zero_slice = 871
(128, 264, 220)
0 50 128 64
Data_slice_shape = (128, 64, 220)
non_zero_slice = 957
(128, 264, 192)
0 50 128 64
Data_slice_shape = (128, 64, 192)
non_zero_slice = 649
(128, 264, 193)
0 50 128 64
Data_slice_shape = (128, 64, 193)
non_zero_slice = 673
(128, 264, 191)
0 50 128 64
Data_slice_shape = (128, 64, 191)
non_zero_slice = 644
(128, 264, 227)
0 50 128 64
Data_slice_shape = (128, 64, 227)
non_zero_slice = 912
(128, 264, 228)
0 50 128 64
Data_slice_shape = (128, 64, 228)
non_zero_slice = 915
(128, 264, 219)
0 50 128 64
Data_slice_shape = (128, 64, 219)
non_zero_slice = 825
(12

In [57]:
def create_dataset(DATA_FOLDER,ML_exp, G1, G2, split_ratio):
    train_combine = []
    eval_combine = []
    test_combine = []
    eval_exp_win_stat = []
    test_exp_win_stat = []
    train_exp_win_stat = []
    for exp in G1:        
        fmt = DATA_FOLDER + 'file_master/' + str(exp) + '.txt'
        a = np.loadtxt(fmt, dtype=str)       
        # for each experiments in G1, we 100% attribute them to training dataset. 
        train_combine = np.concatenate([train_combine, a])
        # housekeeping for the exp/slice included in training
        num_win = int(a[-1].split('win_')[-1][0:1])+1
        for i in range (num_win):
            train_slice = [exp, i]
            train_exp_win_stat.append(train_slice)
            
    for exp in G2:
        fmt = DATA_FOLDER + 'file_master/' + str(exp) + '.txt'
        b = np.loadtxt(fmt, dtype=str)
        #check last element for its window number + 1 to get #of window
        num_win = int(b[-1].split('win_')[-1][0:1])+1
        #check how many time slice per window 
        win_len = len(b)/num_win
        print("num_win",num_win)
        print("len_b",len(b))
        #check randomized which slice would be included in 
        shf_idx = np.arange(num_win)
        print(shf_idx)
        np.random.shuffle(shf_idx)
        print(shf_idx)
        
        # split_eval = split_ratio
        split_eval = num_win//3
        
        #distribute one random window-slice to eval
        for idx in range(split_eval):
            eval_ini = int(shf_idx[idx]*win_len)
            eval_end = int((shf_idx[idx]+1)*win_len)
            eval_combine = np.concatenate([eval_combine,  b[eval_ini:eval_end]])
            # housekeeping for the exp/slice included in evaluating
            eval_slice = [exp,shf_idx[idx]] 
            eval_exp_win_stat.append(eval_slice)
                
        #distribute one random window-slice to test
        test_ini = int(shf_idx[split_eval]*win_len)
        test_end = int((shf_idx[split_eval]+1)*win_len)
        test_combine = np.concatenate([test_combine,  b[test_ini:test_end]])
        # houseckeeping for the exp/slice included in testting
        test_slice = [exp,shf_idx[split_eval]] 
        test_exp_win_stat.append(test_slice)
        
        #distribute the rest of random window-slices to train
        for idx in range(split_eval+1,num_win):
            train_ini = int(shf_idx[idx]*win_len)
            train_end = int((shf_idx[idx]+1)*win_len)
            train_combine = np.concatenate([train_combine, b[train_ini:train_end]])
            # houseckeeping for the exp/slice included in training
            train_slice = [exp,shf_idx[idx]] 
            train_exp_win_stat.append(train_slice)
            
    tot_dataset = len(train_combine) + len(eval_combine) + len(test_combine)
    train_ratio = format(len(train_combine)/tot_dataset, '.2f')
    eval_ratio = format(len(eval_combine)/tot_dataset, '.2f')
    test_ratio = format(len(test_combine)/tot_dataset, '.2f')
    
    stat_dict = {'train_ew': train_exp_win_stat,
                'eval_ew': eval_exp_win_stat,
                'test_ew': test_exp_win_stat,
                'tot_dataset':tot_dataset,
                'train_ratio':train_ratio,
                'eval_ratio':eval_ratio,
                'test_ratio':test_ratio}
    print(stat_dict)
    ML_EXP_PATH = DATA_FOLDER + str(ML_exp)
    if not os.path.exists(ML_EXP_PATH): 
        os.makedirs(ML_EXP_PATH)
    
    with open(ML_EXP_PATH +'/data_stat.txt', 'w') as f:
        print(stat_dict, file=f)
        
    train_data_path = ML_EXP_PATH +  '/train_master.txt'
    np.savetxt(train_data_path, train_combine, fmt="%s")
    
    eval_data_path = ML_EXP_PATH +  '/eval_master.txt'
    np.savetxt(eval_data_path, eval_combine, fmt="%s")
    
    test_data_path = ML_EXP_PATH +  '/test_master.txt'
    np.savetxt(test_data_path, test_combine, fmt="%s")
    return stat_dict

In [58]:
if EXPERIMENT == "clay":
    G1 = ['EB_025_1', 'EB_050_1', 'EB_150_1','PP_025_1','PP_050_1','PP_100_1']
    G2 = ['EB_025_2', 'EB_025_3', 'EB_050_2','EB_050_3','EB_150_2','EB_150_3','PP_025_2','PP_050_2','PP_100_2' ]
    create_dataset(NPY_FOLDER, ML_exp, G1, G2,split_ratio)
else:
    G1 = ['E280_64px_forCNN',
    'E339_64px_forCNN',
    'E347_64px_forCNN',
    'E365_64px_forCNN',
    'E459_64px_forCNN']

    G2 = ['E338_64px_forCNN',
    'E341_64px_forCNN',
    'E342_64px_forCNN',
    'E345_64px_forCNN',
    'E346_64px_forCNN',
    'E355_64px_forCNN',
    'E364_64px_forCNN',
    'E366_64px_forCNN',
    'E369_64px_forCNN',
    'E363_64px_forCNN',
    'E368_64px_forCNN',
    'E458_64px_forCNN',
    'E463_64px_forCNN',
    'E464_64px_forCNN',
    'E499_64px_forCNN']
    create_dataset(NPY_FOLDER, ML_exp, G1, G2,split_ratio)

num_win 5
len_b 1005
[0 1 2 3 4]
[3 1 0 2 4]
num_win 5
len_b 931
[0 1 2 3 4]
[4 1 0 2 3]
num_win 5
len_b 871
[0 1 2 3 4]
[1 4 0 3 2]
num_win 5
len_b 957
[0 1 2 3 4]
[1 3 2 4 0]
num_win 5
len_b 673
[0 1 2 3 4]
[0 1 2 3 4]
num_win 5
len_b 644
[0 1 2 3 4]
[0 1 4 3 2]
num_win 5
len_b 915
[0 1 2 3 4]
[3 0 2 1 4]
num_win 5
len_b 852
[0 1 2 3 4]
[0 4 1 2 3]
num_win 5
len_b 757
[0 1 2 3 4]
[1 3 0 4 2]
{'train_ew': [['EB_025_1', 0], ['EB_025_1', 1], ['EB_025_1', 2], ['EB_025_1', 3], ['EB_025_1', 4], ['EB_050_1', 0], ['EB_050_1', 1], ['EB_050_1', 2], ['EB_050_1', 3], ['EB_050_1', 4], ['EB_150_1', 0], ['EB_150_1', 1], ['EB_150_1', 2], ['EB_150_1', 3], ['EB_150_1', 4], ['PP_025_1', 0], ['PP_025_1', 1], ['PP_025_1', 2], ['PP_025_1', 3], ['PP_025_1', 4], ['PP_050_1', 0], ['PP_050_1', 1], ['PP_050_1', 2], ['PP_050_1', 3], ['PP_050_1', 4], ['PP_100_1', 0], ['PP_100_1', 1], ['PP_100_1', 2], ['PP_100_1', 3], ['PP_100_1', 4], ['EB_025_2', 0], ['EB_025_2', 2], ['EB_025_2', 4], ['EB_025_3', 0], ['EB_025_3'